In [2]:
import mido
import os
from mido import MidiFile, MidiTrack, Message
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [3]:
#specify the path
path='music/classical-piano-type0/mix'


notes = []
for filename in os.listdir(path):
    if filename.endswith(".mid"):
        #print(filename)
        mid = MidiFile(path + '/' + filename)
        for msg in mid:
            if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
                data = msg.bytes()
                notes.append(data[1])

In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

In [5]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 80
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
    
# save a seed to do prediction later
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [6]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath="./checkpoint/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', save_freq=50)

In [7]:
model.fit(np.array(X), np.array(y), 50, 1, verbose=1, callbacks=[model_save_callback])

 50/466 [==>...........................] - ETA: 1:50 - loss: 0.0653
Epoch 00001: saving model to ./checkpoint\checkpoint_model_01.hdf5
100/466 [=====>........................] - ETA: 1:43 - loss: 0.0495
Epoch 00001: saving model to ./checkpoint\checkpoint_model_01.hdf5
150/466 [========>.....................] - ETA: 1:29 - loss: 0.0427
Epoch 00001: saving model to ./checkpoint\checkpoint_model_01.hdf5
200/466 [===========>..................] - ETA: 1:13 - loss: 0.0388
Epoch 00001: saving model to ./checkpoint\checkpoint_model_01.hdf5
250/466 [===============>..............] - ETA: 59s - loss: 0.0361
Epoch 00001: saving model to ./checkpoint\checkpoint_model_01.hdf5
300/466 [==================>...........] - ETA: 45s - loss: 0.0342
Epoch 00001: saving model to ./checkpoint\checkpoint_model_01.hdf5
350/466 [=====================>........] - ETA: 32s - loss: 0.0327
Epoch 00001: saving model to ./checkpoint\checkpoint_model_01.hdf5
400/466 [========================>.....] - ETA: 18s - loss

In [8]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

In [11]:
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 70])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)

newFile = MidiFile()
newFile.tracks.append(track)
newFile.save('mix_epoch1.mid')